# Разбор резюме

Рекрутерам и рекрутерским агенствам присылают резюме в формате pdf и docx. Чтобы вести базу кандидатов, из текстов нужно извлекать имя, возраст, контакты соискателя, предыдущий опыт, желаемую зарплату.

В этом примере рассматриваются только стандартные резюме, сгенерированные сайтом HH. Вариативность таких текстов очень низкая, всё оформлено строго по шаблону. С помощью Yargy-парсера можно написать программу, которая работает с произвольными резюме, но эта задача выходит за рамки данного примера.

![](thumb.png)

# Data

In [1]:
from glob import glob

texts = []
for path in glob('texts/*.txt'):
    with open(path) as file:
        text = file.read()
        texts.append(text)
        
intros = [_[:700] for _ in texts]

In [2]:
from random import seed, sample

seed(41)
for text in sample(intros, 3):
    print(text)
    print('---' * 10)


Мужчина, 46 лет, родился 23 марта 1970

Проживает: Владикавказ
Гражданство: Россия, есть разрешение на работу: Россия
Не готов к переезду, не готов к командировкам

Желаемая должность и зарплата
дизайнер
Маркетинг, реклама, PR

• Печатная реклама
• Арт директор
• Дизайнер

Занятость: полная занятость, частичная занятость
График работы: полный день, гибкий график, удаленная работа
Желательное время в пути до работы: не имеет значения

25 000
руб.

Опыт работы — 24 года 10 месяцев
Январь 2007 —
настоящее время
9 лет 9 месяцев

Индивидуальное предпринимательство / частная практика /
фриланс
www.weblancer.net/users/voenerg/portfolio/

дизайнер
Разработка айдентики, POS-материалов, наружной рекл
------------------------------

Женщина, 25 лет, родилась 25 апреля 1991

Проживает: Казань, м. Суконная слобода
Гражданство: Россия, есть разрешение на работу: Россия
Готова к переезду, готова к командировкам

Желаемая должность и зарплата
Менеджер-экономист
Бухгалтерия, управленческий учет, финан

# Grammar

In [3]:
import json

from ipymarkup import (
    AsciiMarkup,
    show_markup as show_markup_
)

from yargy import (
    Parser,
    rule, or_, and_, not_
)
from yargy.predicates import (
    eq, in_,
    type, normalized,
    dictionary,
    gte, lte
)
from yargy.pipelines import (
    pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy.tokenizer import MorphTokenizer, EOL


Intro = fact(
    'Intro',
    ['gender', 'age', 'birth', 'location',
     attribute('citizenship').repeatable(),
     attribute('permission').repeatable(),
     'relocation', 'travel',
     'position',
     attribute('subspecializations').repeatable(),
     'employment', 'schedule', 'commute',
     'salary'
    ]
)


INT = type('INT')
COMMA = eq(',')
COLON = eq(':')


def show_markup(text, spans):
    show_markup_(text, spans, AsciiMarkup)

        
def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))


def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        spans = [_.span for _ in matches]
        show_markup(line, spans)

## Gender

In [4]:
GENDERS = {
    'Женщина': 'female',
    'Мужчина': 'male'
}

GENDER = in_(GENDERS).interpretation(
    Intro.gender.custom(GENDERS.get)
)


show_matches(
    GENDER,
    'мужчина, Мужчина, мужчину',
    'Женщина'
)

мужчина, Мужчина, мужчину
         -------         


Женщина
-------


## Age

In [5]:
AGE = rule(
    INT,
    normalized('год')
)


show_matches(
    AGE,
    '21 год, 25 лет'
)

21 год, 25 лет
------  ------


## Birth

In [6]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)


AGE = rule(
    INT.interpretation(
        Intro.age.custom(int)
    ),
    normalized('год')
)

MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

YEAR = and_(
    gte(1900),
    lte(2100)
).interpretation(
    Date.year.custom(int)
)

DATE = rule(
    DAY,
    MONTH_NAME,
    YEAR
).interpretation(
    Date
)

BIRTH = rule(
    normalized('родиться'),
    DATE.interpretation(
        Intro.birth
    )
)


show_matches(
    BIRTH,
    'родился 21 февраля 1990',
    'родиться 32 сентябрь 2000',
    'родилась 01 июля 1917',
)

родился 21 февраля 1990
-----------------------


родиться 32 сентябрь 2000


родилась 01 июля 1917
---------------------


## Socdem

In [7]:
SOCDEM = rule(
    GENDER, COMMA,
    AGE, COMMA,
    BIRTH
)


parser = Parser(SOCDEM)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])

Мужчина, 29 лет, родился 17 ноября 1986
Женщина, 29 лет, родилась 2 марта 1987
Мужчина, 34 года, родился 14 августа 1982
Женщина, 36 лет, родилась 20 апреля 1980
Мужчина, 45 лет, родился 16 марта 1971
Женщина, 38 лет, родилась 2 мая 1978
Женщина, 33 года, родилась 5 октября 1982
Женщина, 26 лет, родилась 15 апреля 1990
Женщина, 29 лет, родилась 14 мая 1987
Мужчина, 33 года, родился 14 января 1983


## Location

In [8]:
# https://api.hh.ru/metro
# https://api.hh.ru/areas

def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')
            
            
METRO_STATIONS = set(load_lines('dicts/metro.txt'))
AREAS = set(load_lines('dicts/areas.txt'))
seed(10)
sample(sorted(METRO_STATIONS), 10), sample(sorted(AREAS), 10)

(['Проспект Мира',
  'Алма-Атинская',
  'Нахимовский проспект',
  'Парк Победы',
  'Проспект свободы',
  'Академгородок',
  'Достоевская',
  'Окружная',
  'Партизанская',
  'Козья слобода'],
 ['Старонижестеблиевская',
  'Голицыно',
  'Атаманская',
  'Перевальск',
  'Обоянь',
  'Кораблино',
  'Биробиджан',
  'Ижевск',
  'Чаплыгин',
  'Куйбышево'])

In [9]:
Location = fact(
    'Location',
    ['area', 'metro']
)


METRO = rule(
    'м', '.',
    pipeline(METRO_STATIONS).interpretation(
        Location.metro
    )
)

AREA = pipeline(AREAS).interpretation(
    Location.area
)

LOCATION = rule(
    AREA,
    rule(
        COMMA,
        METRO
    ).optional()
).interpretation(
    Location
)


show_matches(
    LOCATION,
    'место проживания: Москва, м. Парк Победы',
    'Киев, м.Киевская',
    'Россия',
    'в Москве',
    'м. парк победы',
    'на м. Кропоткинской',
)

место проживания: Москва, м. Парк Победы
                  ----------------------


Киев, м.Киевская
----------------


Россия
------


в Москве


м. парк победы


на м. Кропоткинской


In [10]:
TITLE = rule(
    normalized('проживает'), COLON
)

LIVES_AT = rule(
    TITLE,
    LOCATION
)


parser = Parser(LIVES_AT)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])

Проживает: Санкт-Петербург
Проживает: Вологда
Проживает: Новосибирск, м. площадь Карла Маркса
Проживает: Йошкар-Ола
Проживает: Тюмень
Проживает: Туапсе
Проживает: Нефтеюганск
Проживает: Москва
Проживает: Псков
Проживает: Омск


## Citizenship

In [11]:
TITLE = rule(
    'Гражданство', COLON
)

ITEM = AREA.interpretation(
    Intro.citizenship
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional()
)

CITIZENSHIP = rule(
    TITLE,
    LOCATIONS
)


show_matches(
    CITIZENSHIP,
    'Гражданство: Россия, Франция',
    'Гражданство: Россия, Франция, Украина',
)

Гражданство: Россия, Франция
----------------------------


Гражданство: Россия, Франция, Украина
----------------------------         


## Permission

In [12]:
TITLE = pipeline([
    'есть разрешение на работу:'
])

ITEM = AREA.interpretation(
    Intro.permission
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional().repeatable()
)

PERMISSION = rule(
    TITLE,
    LOCATIONS
)


show_matches(
    PERMISSION,
    'есть разрешение на работу: Россия, Франция, Украина',
    'есть разрешение на работу: Россия',
)

есть разрешение на работу: Россия, Франция, Украина
---------------------------------------------------


есть разрешение на работу: Россия
---------------------------------


## Relocation

In [13]:
Relocation = fact(
    'Relocation',
    ['ready', attribute('where').repeatable()]
)

TYPES = {
    'готов к переезду': True,
    'хочу переехать': True,
    'не готов к переезду': False
}

IS_READY = morph_pipeline(TYPES).interpretation(
    Relocation.ready.normalized().custom(TYPES.get)
)

ITEM = AREA.interpretation(
    Relocation.where
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional().repeatable()
)

RELOCATION = rule(
    IS_READY,
    rule(
        COLON,
        LOCATIONS
    ).optional()
).interpretation(
    Relocation
).interpretation(
    Intro.relocation
)


show_matches(
    RELOCATION,
    'готов к переезду',
    'не готова к переезду',
    'готова к переезду: Россия, Украина, СНГ',
    'хочу переехать: Франция',
)

готов к переезду
----------------


не готова к переезду
--------------------


готова к переезду: Россия, Украина, СНГ
----------------------------------     


хочу переехать: Франция
-----------------------


## Travel

In [14]:
TYPES = {
    'готов к командировкам': True,
    'готов к редким командировкам': True,
    'не готов к командировкам': False
}

TRAVEL = morph_pipeline(TYPES).interpretation(
    Intro.travel.normalized().custom(TYPES.get)
)


show_matches(
    TRAVEL,
    'готова к командировкам',
    'не готов к редким командировкам',
)

готова к командировкам
----------------------


не готов к редким командировкам
   ----------------------------


## Position

In [15]:
# https://api.hh.ru/specializations

SPECIALIZATIONS = set(load_lines('dicts/specialization.txt'))
SUBSPECIALIZATIONS = set(load_lines('dicts/subspecialization.txt'))

seed(10)
sample(sorted(SPECIALIZATIONS), 10), sample(sorted(SUBSPECIALIZATIONS), 10)

(['Продажи',
  'Административный персонал',
  'Консультирование',
  'Медицина, фармацевтика',
  'Юристы',
  'Автомобильный бизнес',
  'Государственная служба, некоммерческие организации',
  'Маркетинг, реклама, PR',
  'Транспорт, логистика',
  'Домашний персонал'],
 ['Риски: рыночные',
  'Тренерский состав',
  'Делопроизводство',
  'Администратор баз данных',
  'Планирование, Размещение рекламы',
  'Охранник',
  'Кредиты: розничные',
  'Банкеты',
  'Информационные технологии, Интернет, Мультимедиа',
  'Электроэнергетика'])

In [16]:
TITLE = pipeline([
    'Желаемая должность и зарплата'
])

DOT = eq('•')

SUBTITLE = not_(DOT).repeatable().interpretation(
    Intro.position
)

SPECIALIZATION = pipeline(SPECIALIZATIONS)

SUBSPECIALIZATION = pipeline(SUBSPECIALIZATIONS)

ITEM = rule(
    DOT,
    or_(
        SPECIALIZATION,
        SUBSPECIALIZATION
    ).interpretation(
        Intro.subspecializations
    )
)

POSITION = rule(
    TITLE,
    SUBTITLE,
    ITEM.repeatable()
)


TOKENIZER = MorphTokenizer().remove_types(EOL)


parser = Parser(POSITION, tokenizer=TOKENIZER)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])
        print('---')

Желаемая должность и зарплата
Супервайзер
Продажи

• Дистрибуция
• Прямые продажи
• Продукты питания
---
Желаемая должность и зарплата
Продавец-консультант
Продажи

• Розничная торговля
• Торговые сети
• Продавец в магазине
---
Желаемая должность и зарплата
Инженер по строительству  сотовой связи
Информационные технологии, интернет, телеком

• Передача данных и доступ в интернет
• Сотовые, Беспроводные технологии
• Телекоммуникации
---
Желаемая должность и зарплата
Главный бухгалтер, заместитель главного бухгалтера
Бухгалтерия, управленческий учет, финансы предприятия

• Руководство бухгалтерией
---
Желаемая должность и зарплата
Инженер по транспорту
Транспорт, логистика

• Автоперевозки
---
Желаемая должность и зарплата
Директор магазина, заместитель директора
Продажи

• Розничная торговля
• Торговые сети
---
Желаемая должность и зарплата
Лаборант химического анализа
Добыча сырья

• Нефть
• Газ
---
Желаемая должность и зарплата
Администратор;  Помощник бухгалтера
Бухгалтерия, управлен

## Employment

In [17]:
TITLE = rule(
    'Занятость', COLON
)

TYPES = {
    'полная': 'full',
    'полная занятость': 'full',
    'частичная': 'part',
    'частичная занятость': 'part',
    'волонтерство': 'volunteer',
    'стажировка': 'intern',
    'проектная работа': 'project'
    
}

TYPE = pipeline(TYPES).interpretation(
    Intro.employment.normalized().custom(TYPES.get)
)

TYPES = rule(
    TYPE,
    rule(
        COMMA,
        TYPE
    ).optional().repeatable()
)

EMPLOYMENT = rule(
    TITLE,
    TYPES
)


show_matches(
    EMPLOYMENT,
    'Занятость: полная, частичная',
    'Занятость: стажировка',
)

Занятость: полная, частичная
----------------------------


Занятость: стажировка
---------------------


## Schedule

In [18]:
TITLE = pipeline([
    'График работы:'
])

TYPES = {
    'полный день': 'full',
    'сменный график': 'part',
    'вахтовый метод': 'vahta',
    'гибкий график': 'flex',
    'удаленная работа': 'remote',
    'стажировка': 'intern'
}

TYPE = morph_pipeline(TYPES).interpretation(
    Intro.schedule.normalized().custom(TYPES.get)
)

TYPES = rule(
    TYPE,
    rule(
        COMMA,
        TYPE
    ).optional().repeatable()
)

SCHEDULE = rule(
    TITLE,
    TYPES
)


show_matches(
    SCHEDULE,
    'График работы: полный день, удаленная работа',
    'График работы: стажировка',
)

График работы: полный день, удаленная работа
--------------------------------------------


График работы: стажировка
-------------------------


## Commute

In [19]:
TITLE = pipeline([
    'Желательное время в пути до работы:',
])

TYPES = {
    'не более часа': '<1h',
    'не имеет значения': 'any',
    'не более полутора часов': '<1h30m'
}

TYPE = pipeline(TYPES).interpretation(
    Intro.commute.normalized().custom(TYPES.get)
)

COMMUTE = rule(
    TITLE,
    TYPE
)


show_matches(
    COMMUTE,
    'Желательное время в пути до работы: не более часа',
    'Желательное время в пути до работы: не имеет значения',
)

Желательное время в пути до работы: не более часа
-------------------------------------------------


Желательное время в пути до работы: не имеет значения
-----------------------------------------------------


## Money

In [20]:
Money = fact(
    'Money',
    ['amount', 'currency']
)


CURRENCIES = {
    'руб.': 'RUB',
    'грн.': 'GRN',
    'бел. руб.': 'BEL',
    'RUB': 'RUB',
    'EUR': 'EUR',
    'KZT': 'KZT',
    'USD': 'USD',
    'KGS': 'KGS'
}

CURRENCY = pipeline(CURRENCIES).interpretation(
    Money.currency.normalized().custom(CURRENCIES.get)
)


def normalize_amount(value):
    return int(value.replace(' ', ''))


AMOUNT = or_(
    rule(INT),
    rule(INT, INT),
).interpretation(
    Money.amount.custom(normalize_amount)
)

MONEY = rule(
    AMOUNT,
    CURRENCY
).interpretation(
    Money
).interpretation(
    Intro.salary
)


show_matches(
    MONEY,
    '1 500 руб.',
    '1 000 000 грн.',
    '5000 бел.руб.',
    '20 000 KGS',
)

1 500 руб.
----------


1 000 000 грн.
  ------------


5000 бел.руб.
-------------


20 000 KGS
----------


## Intro

In [21]:
INTRO = rule(
    SOCDEM,
    LIVES_AT,
    CITIZENSHIP, COMMA, PERMISSION,
    RELOCATION, COMMA, TRAVEL,
    POSITION,
    EMPLOYMENT,
    SCHEDULE,
    COMMUTE,
    MONEY
).interpretation(
    Intro
)


parser = Parser(INTRO, tokenizer=TOKENIZER)
seed(10)
for text in sample(intros, 10):
    matches = list(parser.findall(text))
    if matches:
        match = matches[0]
        fact = match.fact
        show_markup(text, fact.spans)
        show_json(fact.as_json)

 
Мужчина, 29 лет, родился 17 ноября 1986 
-------  --              -- ------ ---- 
 
Проживает: Санкт-Петербург 
Гражданство: Россия, есть разрешение на работу: Россия 
             ------                             ------ 
Не готов к переезду, не готов к командировкам 
-------------------  ------------------------ 
 
Желаемая должность и зарплата 
Супервайзер 
------------
Продажи 
------- 
 
• Дистрибуция 
  ----------- 
• Прямые продажи 
  -------------- 
• Продукты питания 
  ---------------- 
 
Занятость: полная занятость 
           ---------------- 
График работы: полный день 
               ----------- 
Желательное время в пути до работы: не имеет значения 
                                    ----------------- 
 
40 000 
------ 
руб. 
---- 
 
Опыт работы — 7 лет 8 месяцев 
Февраль 2016 — 
Июнь 2016 
5 месяцев 
 
ЗАО Бастион/ООО ДК Северозапад 
Санкт-Петербург 
 
Менеджер 
Продажа стабилизаторов напряжения и компрессорного 
оборудования.Повысил навыки продаж, 
делового общения

{
  "gender": "male",
  "age": 29,
  "birth": {
    "year": 1986,
    "month": 11,
    "day": 17
  },
  "citizenship": [
    "Россия"
  ],
  "permission": [
    "Россия"
  ],
  "relocation": {
    "ready": false,
    "where": []
  },
  "travel": false,
  "position": "Супервайзер Продажи",
  "subspecializations": [
    "Дистрибуция",
    "Прямые продажи",
    "Продукты питания"
  ],
  "employment": "full",
  "schedule": "full",
  "commute": "any",
  "salary": {
    "amount": 40000,
    "currency": "RUB"
  }
}


 
Женщина, 29 лет, родилась 2 марта 1987 
-------  --               - ----- ---- 
 
Проживает: Вологда 
Гражданство: Россия, есть разрешение на работу: Россия 
             ------                             ------ 
Не готова к переезду, не готова к командировкам 
--------------------  ------------------------- 
 
Желаемая должность и зарплата 
Продавец-консультант 
---------------------
Продажи 
------- 
 
• Розничная торговля 
  ------------------ 
• Торговые сети 
  ------------- 
• Продавец в магазине 
  ------------------- 
 
Занятость: полная занятость 
           ---------------- 
График работы: полный день 
               ----------- 
Желательное время в пути до работы: не имеет значения 
                                    ----------------- 
 
24 000 
------ 
руб. 
---- 
 
Опыт работы — 1 год 9 месяцев 
Ноябрь 2014 — 
Июль 2016 
1 год 9 месяцев 
 
ООО"Келли Сервисез Си-Ай-эс" опыт работы 2 года, 
Вологда 
 
Специалист по работе с клиентами 
обработка входящих вызовов,консульта

{
  "gender": "female",
  "age": 29,
  "birth": {
    "year": 1987,
    "month": 3,
    "day": 2
  },
  "citizenship": [
    "Россия"
  ],
  "permission": [
    "Россия"
  ],
  "relocation": {
    "ready": false,
    "where": []
  },
  "travel": false,
  "position": "Продавец-консультант Продажи",
  "subspecializations": [
    "Розничная торговля",
    "Торговые сети",
    "Продавец в магазине"
  ],
  "employment": "full",
  "schedule": "full",
  "commute": "any",
  "salary": {
    "amount": 24000,
    "currency": "RUB"
  }
}


 
Мужчина, 34 года, родился 14 августа 1982 
-------  --               -- ------- ---- 
 
Проживает: Новосибирск, м. площадь Карла Маркса 
Гражданство: Россия, есть разрешение на работу: Россия 
             ------                             ------ 
Не готов к переезду, готов к командировкам 
-------------------  --------------------- 
 
Желаемая должность и зарплата 
Инженер по строительству  сотовой связи 
----------------------------------------
Информационные технологии, интернет, телеком 
-------------------------------------------- 
 
• Передача данных и доступ в интернет 
  ----------------------------------- 
• Сотовые, Беспроводные технологии 
  -------------------------------- 
• Телекоммуникации 
  ---------------- 
 
Занятость: полная занятость, частичная занятость 
                             ------------------- 
График работы: полный день 
               ----------- 
Желательное время в пути до работы: не более часа 
                                    ------------- 
 


...

{
  "gender": "female",
  "age": 33,
  "birth": {
    "year": 1982,
    "month": 10,
    "day": 5
  },
  "citizenship": [
    "Россия"
  ],
  "permission": [
    "Россия"
  ],
  "relocation": {
    "ready": false,
    "where": []
  },
  "travel": false,
  "position": "Лаборант химического анализа Добыча сырья",
  "subspecializations": [
    "Нефть",
    "Газ"
  ],
  "employment": "full",
  "schedule": "full",
  "commute": "any",
  "salary": {
    "amount": 35000,
    "currency": "RUB"
  }
}


 
Женщина, 26 лет, родилась 15 апреля 1990 
-------  --               -- ------ ---- 
 
Проживает: Москва 
Гражданство: Россия, есть разрешение на работу: Россия 
             ------                             ------ 
Не готова к переезду, не готова к командировкам 
--------------------  ------------------------- 
 
Желаемая должность и зарплата 
Администратор;  Помощник бухгалтера 
------------------------------------
Бухгалтерия, управленческий учет, финансы предприятия 
----------------------------------------------------- 
 
• Учет счетов и платежей 
  ---------------------- 
• Казначейство 
  ------------ 
• Кассир, Инкассатор 
  ------------------ 
 
Занятость: полная занятость 
           ---------------- 
График работы: полный день 
               ----------- 
Желательное время в пути до работы: не имеет значения 
                                    ----------------- 
 
35 000 
------ 
руб. 
---- 
 
Опыт работы — 7 лет 1 месяц 
Сентябрь 2012 — 
Сентябрь 2015 
3 года 1 месяц 
 

{
  "gender": "female",
  "age": 26,
  "birth": {
    "year": 1990,
    "month": 4,
    "day": 15
  },
  "citizenship": [
    "Россия"
  ],
  "permission": [
    "Россия"
  ],
  "relocation": {
    "ready": false,
    "where": []
  },
  "travel": false,
  "position": "Администратор; Помощник бухгалтера Бухгалтерия, управленческий учет, финансы предприятия",
  "subspecializations": [
    "Учет счетов и платежей",
    "Казначейство",
    "Кассир, Инкассатор"
  ],
  "employment": "full",
  "schedule": "full",
  "commute": "any",
  "salary": {
    "amount": 35000,
    "currency": "RUB"
  }
}


 
Женщина, 29 лет, родилась 14 мая 1987 
-------  --               -- --- ---- 
 
Проживает: Псков 
Гражданство: Россия, есть разрешение на работу: Россия 
             ------                             ------ 
Не готова к переезду, готова к редким командировкам 
--------------------  ----------------------------- 
 
Желаемая должность и зарплата 
мастер кузовного цеха 
----------------------
Автомобильный бизнес 
-------------------- 
 
• Сервисное обслуживание 
  ---------------------- 
• Автозапчасти 
  ------------ 
• Автожестянщик 
  ------------- 
 
Занятость: полная занятость 
           ---------------- 
График работы: полный день 
               ----------- 
Желательное время в пути до работы: не имеет значения 
                                    ----------------- 
 
30 000 
------ 
руб. 
---- 
 
Опыт работы — 8 лет 7 месяцев 
Март 2008 — 
настоящее время 
8 лет 7 месяцев 
 
ООО "Автопремиум" 
Псков 
 
Руководитель МКЦ 
Контроль качество Кузовного ремонта организация работы,

{
  "gender": "female",
  "age": 29,
  "birth": {
    "year": 1987,
    "month": 5,
    "day": 14
  },
  "citizenship": [
    "Россия"
  ],
  "permission": [
    "Россия"
  ],
  "relocation": {
    "ready": false,
    "where": []
  },
  "travel": true,
  "position": "мастер кузовного цеха Автомобильный бизнес",
  "subspecializations": [
    "Сервисное обслуживание",
    "Автозапчасти",
    "Автожестянщик"
  ],
  "employment": "full",
  "schedule": "full",
  "commute": "any",
  "salary": {
    "amount": 30000,
    "currency": "RUB"
  }
}
